### Objective：

Implement a special Mod class to implement some concepts in modular arithmetic.

---

### Functionalities and characteristics:

- Assume **n** is a positive integer and **a**, **b** are integers
- **a** and **b** are congruent modulo if a%n = b%n


---
### Implementation:

> Before refactoring

In [1]:
class Mod:
    
    def __init__(self, value, modulus):
        if not isinstance(value, int):
            raise TypeError('Value must be an integer.')
        if not isinstance(modulus, int):
            raise TypeError('Modulus must be an integer.')
        if modulus < 0:
            raise ValueError('Modulus must be positive.')
        
        self._modulus = modulus
        self._value = value % self._modulus
        
    @property
    def value(self):
        return self._value
    
    @property
    def modulus(self):
        return self._modulus
    
    def __repr__(self):
        return 'Mod({}, {})'.format(self.value, self.modulus)
    
    def __int__(self):
        return self.value
    
    def __lt__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) < (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) < (other % self._modulus)
        return NotImplemented
    
    def __le__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) <= (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) <= (other % self._modulus)
        return NotImplemented
    
    def __gt__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) > (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) > (other % self._modulus)
        return NotImplemented
    
    def __ge__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) >= (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) >= (other % self._modulus)
        return NotImplemented
    
    def __ne__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) != (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) != (other % self._modulus)
        return NotImplemented
    
    def __eq__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            return (self._value % self._modulus) == (other._value % other._modulus)
        if isinstance(other, int):
            return (self._value % self._modulus) == (other % self._modulus)
        return NotImplemented
    
    def __hash__(self):
        return hash((self.value, self.modulus))

    def __add__(self, other):
        if isinstance(other, Mod) and self.modulus == other.modulus:
            return Mod(self.value + other.value, self.modulus)
        if isinstance(other, int):
            return Mod(self.value + other, self.modulus)
        return NotImplemented
    
    def __sub__(self, other):
        if isinstance(other, Mod) and self.modulus == other.modulus:
            return Mod(self.value - other.value, self.modulus)
        if isinstance(other, int):
            return Mod(self.value - other, self.modulus)
        return NotImplemented
    
    def __mul__(self, other):
        if isinstance(other, Mod) and self.modulus == other.modulus:
            return Mod(self.value * other.value, self.modulus)
        if isinstance(other, int):
            return Mod(self.value * other, self.modulus)
        return NotImplemented
    
    def __pow__(self, other):
        if isinstance(other, Mod) and self.modulus == other.modulus:
            return Mod(self.value ** other.value, self.modulus)
        if isinstance(other, int):
            return Mod(self.value ** (other % self.modulus), self.modulus)
        return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            self._value = (self.value + other.value) % self.modulus
            return self
        if isinstance(other, int):
            self._value = (self.value + other) % self.modulus
            return self
        return NotImplemented
    
    def __isub__(self):
        if isinstance(other, Mod) and self._modulus == other._modulus:
            self._value = (self.value - other.value) % self.modulus
            return self
        if isinstance(other, int):
            self._value = (self.value - other) % self.modulus
            return self
        return NotImplemented
    
    def __imul__(self):
        if isinstance(other, Mod) and self.modulus == other._modulus:
            self._value = (self.value * other.value) % self.modulus 
            return self
        if isinstance(other, int):
            self._value = (self.value * other) % self.modulus
            return self
        return NotImplemented
    
    def __ipow__(self):
        if isinstance(other, Mod) and self.modulus == other.modulus:
            self._value = (self.value ** other.value) % self.modulus 
            return self
        if isinstance(other, int):
            self._value = (self.value ** (other % self.modulus)) % self.modulus
            return self
        return NotImplemented

In [2]:
m1 = Mod(8, 3)
m2 = Mod(9, 3)

In [3]:
int(m1)

2

In [4]:
m1 + m2

Mod(2, 3)

In [5]:
m1 ** m2

Mod(1, 3)

In [6]:
Mod(2, 3) * 18

Mod(0, 3)

> After refactoring

In [7]:
from functools import total_ordering
import operator

In [8]:
@total_ordering
class Mod:
    
    def __init__(self, value, modulus):
        if not isinstance(value, int):
            raise TypeError('Value must be an integer.')
        if not isinstance(modulus, int):
            raise TypeError('Modulus must be an integer.')
        if modulus < 0:
            raise ValueError('Modulus must be positive.')
        
        self._modulus = modulus
        self._value = value % self._modulus
        
    @property
    def value(self):
        return self._value
    
    @property
    def modulus(self):
        return self._modulus
    
    def __repr__(self):
        return 'Mod({}, {})'.format(self.value, self.modulus)
    
    def __int__(self):
        return self.value
    
    def _get_value(self, other):
        if isinstance(other, int):
            return other % self.modulus
        if isinstance(other, Mod) and self.modulus == other.modulus:
            return other.value
        raise TypeError('Incompatible types.')
        
    def _perform_operation(self, other, op, *, in_place=False):
        other_value = self._get_value(other)
        new_value = op(self.value, other_value)
        if in_place:
            self.value = new_value % self.modulus
            return self
        else:
            return Mod(new_value, self.modulus)
    
    def __lt__(self, other):
        try:
            return self.value < _get_value(self, other)
        except TypeError:
            return NotImplemented
    
    def __eq__(self, other):
        try:
            return self.value == _get_value(self, other)
        except TypeError:
            return NotImplemented

    def __hash__(self):
        return hash((self.value, self.modulus))

    def __add__(self, other):
        return self._perform_operation(other, operator.add)
    
    def __sub__(self, other):
        return self._perform_operation(other, operator.sub)
    
    def __mul__(self, other):
        return self._perform_operation(other, operator.mul)
    
    def __pow__(self, other):
        return self._perform_operation(other, operator.pow)
    
    def __iadd__(self, other):
        return self._perform_operation(other, operator.add, in_place=True)
    
    def __isub__(self):
        return self._perform_operation(other, operator.sub, in_place=True)
    
    def __imul__(self):
        return self._perform_operation(other, operator.mul, in_place=True)
    
    def __ipow__(self):
        return self._perform_operation(other, operator.pow, in_place=True)

In [9]:
m1 = Mod(8, 3)
m2 = Mod(9, 3)

In [10]:
int(m1)

2

In [11]:
m1 + m2

Mod(2, 3)

In [12]:
m1 ** m2

Mod(1, 3)

In [13]:
Mod(2, 3) * 18

Mod(0, 3)